In [1]:
import numpy as np
import xtrack as xt         
import xpart as xp
import xdeps as xd   
import inspect
import pickle
import scipy

In [2]:
with open('knob.pkl', 'rb') as f:
    knobs = pickle.load(f)

knobs

{'kqtf.b2': 0.001208677013518323,
 'kqtd.b2': 0.00038331555223907116,
 'ksf.b2': 0.013659436695399586,
 'ksd.b2': -0.027437069853417304,
 'kqt13.l2b2': 0.0004514499114322108,
 'kqt12.l2b2': -0.003255459636063764,
 'kqtl11.l2b2': 0.003621832707128442,
 'kq10.l2b2': 0.007038061942052969,
 'kq9.l2b2': -0.006933724300000922,
 'kq8.l2b2': 0.007049750740201287,
 'kq7.l2b2': -0.007838443884785605,
 'kq6.l2b2': 0.0043085171149305265,
 'kq5.l2b2': -0.003279698116036019,
 'kq4.l2b2': 0.0019927108461440654,
 'kqx.l2': 0.008755083323428548,
 'kq4.r2b2': -0.003630665367308744,
 'kq5.r2b2': 0.0048330772584889384,
 'kq6.r2b2': -0.003938045240378545,
 'kq7.r2b2': 0.00749567152642893,
 'kq8.r2b2': -0.0056231039407293225,
 'kq9.r2b2': 0.0067684978060658105,
 'kq10.r2b2': -0.005326779941068593,
 'kqtl11.r2b2': 0.0004494694925349877,
 'kqt12.r2b2': -0.0017493728475043034,
 'kqt13.r2b2': 0.004448848638839975,
 'kqt13.l4b2': -0.003479263576853937,
 'kqt12.l4b2': 0.0023715956762976787,
 'kqtl11.l4b2': -3.700

In [21]:
line_path = '../bkp_HL/HL_IR7_rematched/b4_sequence.json'
#line_path = '../HL_IR7_IR3_rematched/b4_sequence_patched.json'
line = xt.Line.from_json(line_path)

Loading line from dict:   0%|          | 0/148580 [00:00<?, ?it/s]

Done loading line from dict.           


In [19]:
for k, v in knobs.items():
    line.vars[k] = v

In [22]:
line.vars.get_table()


Table: 1 row, 2 cols
name     value
t_turn_s     0

In [20]:
line.vv['kqtf.b2']

0.001208677013518323

In [20]:
save = False
if save:
    line.to_json('HL_phase_advance_rematched.json')

In [6]:
"""speed = line.particle_ref._xobject.beta0[0]*scipy.constants.c
harmonic_number = 35640
voltage = 12e6/len(line.get_elements_of_type(xt.Cavity)[1])
frequency = harmonic_number * speed /line.get_length()
for side in ['l', 'r']:
    for cell in ['a','b','c','d']:
        line[f'acsca.{cell}5{side}4.b2'].voltage = voltage
        line[f'acsca.{cell}5{side}4.b2'].frequency = frequency"""


In [21]:
end_s = line.get_length()

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
TCLA_name = 'tcla.a5l3.b2'
TCP_name = 'tcp.d6r7.b2'

TCCS_loc = end_s - 6773.7 #6775
TCCP_loc = end_s - 6653.3 #6655
TARGET_loc = end_s - (6653.3 +  0.070/2 + 0.005/2)
TCLA_loc = line.get_s_position()[line.element_names.index(TCLA_name)]


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name='tccs.5r3.b2')
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccs.5r3.b2_aper')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name='tccp.4l3.b2')
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccp.4l3.b2_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name='target.4l3.b2')
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='target.4l3.b2_aper')

In [22]:
tw = line.twiss(method='4d')

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [23]:
df = tw.to_pandas()

In [24]:
df0 = df[(df['name']==TCCS_name) |  (df['name']==TCP_name)]# TCP_name]] 
df0

,name,s,x,px,y,py,zeta,delta,ptau,W_matrix,...,bx_chrom,by_chrom,ax_chrom,ay_chrom,wx_chrom,wy_chrom,ddx,ddpx,ddy,ddpy
35674,tcp.d6r7.b2,6459.742798,1.473661e-06,-1.526910e-08,4.864029e-08,3.687746e-09,0.000016,0.0,0.0,"[[19.481290519519053, -1.7763568394002505e-15,...",...,0.768490,75.066566,4.196799,-17.407362,4.266579,77.058455,64.692829,-0.645058,30.174357,0.571277
110258,tccs.5r3.b2,19885.183171,-7.405086e-07,-1.251547e-08,5.168134e-07,-8.295849e-10,0.000016,0.0,0.0,"[[9.062889737685175, 4.440892098500626e-16, -0...",...,-72.502243,8.160270,-30.578080,-65.028132,78.686684,65.538141,-17.161598,-0.429326,-9.379823,0.117877


In [25]:
optphase = 158.521582158919/360
optphase

0.4403377282192194

In [26]:
(df0['muy'].iloc[1] - df0['muy'].iloc[0])%1

0.4188368424826301

0.4188691186446931

In [25]:
path = '/afs/cern.ch/work/b/bjlindst/public/git/madx/hllhc/generate_xtrack/out/250muradXing20cmBetaNoTCLD-ORBITBUMP-TCPBETARETUNE-OFFSETAPERir7re12c6/'
#path = '/afs/cern.ch/work/b/bjlindst/public/git/madx/hllhc/ir7_match/'
lhc = xt.line.Line.from_json(path+"b4_sequence.json")

Loading line from dict:   0%|          | 0/147676 [00:00<?, ?it/s]

Done loading line from dict.           


In [26]:
lhc.vars['kqtf.b2']._find_dependant_targets()

[vars['kqtf.b2'],
 vars['kqtf.a67b2'],
 element_refs['mqt.15r6.b2..2'].knl,
 element_refs['mqt.15r6.b2..2'].knl[1],
 element_refs['mqt.15r6.b2..2'],
 element_refs['mqt.15r6.b2..1'],
 element_refs['mqt.15r6.b2..1'].knl,
 element_refs['mqt.15r6.b2..1'].knl[1],
 element_refs['mqt.17r6.b2..2'].knl[1],
 element_refs['mqt.17r6.b2..2'].knl,
 element_refs['mqt.17r6.b2..2'],
 element_refs['mqt.17r6.b2..1'],
 element_refs['mqt.17r6.b2..1'].knl,
 element_refs['mqt.17r6.b2..1'].knl[1],
 element_refs['mqt.19r6.b2..2'].knl[1],
 element_refs['mqt.19r6.b2..2'],
 element_refs['mqt.19r6.b2..2'].knl,
 element_refs['mqt.19r6.b2..1'].knl[1],
 element_refs['mqt.19r6.b2..1'],
 element_refs['mqt.19r6.b2..1'].knl,
 element_refs['mqt.21r6.b2..2'],
 element_refs['mqt.21r6.b2..2'].knl[1],
 element_refs['mqt.21r6.b2..2'].knl,
 element_refs['mqt.21r6.b2..1'].knl[1],
 element_refs['mqt.21r6.b2..1'],
 element_refs['mqt.21r6.b2..1'].knl,
 element_refs['mqt.21l7.b2..2'],
 element_refs['mqt.21l7.b2..2'].knl,
 element_re

In [14]:
line['mqt.15r6.b2..2']

Multipole(_order=1, inv_factorial_order=1.0, length=0.16, hxl=0.0, hyl=0.0, radiation_flag=0, delta_taper=0.0, knl=[0.         0.00015886], ksl=[0. 0.], _internal_record_id=RecordIdentifier(buffer_id=0, offset=0))

In [9]:
#path_1 = '/afs/cern.ch/work/b/bjlindst/public/git/madx/hllhc/generate_xtrack/out/250muradXing20cmBetaNoTCLD-ORBITBUMP-TCPBETARETUNE-OFFSETAPERir7re12c6/'
path_1 = '/afs/cern.ch/work/b/bjlindst/public/git/madx/hllhc/generate_xtrack/out/250muradXing20cmBetaNoTCLD-ORBITBUMP-TCPBETARETUNE-OFFSETAPERir7re12c6/'

#path = '/afs/cern.ch/work/b/bjlindst/public/git/madx/hllhc/ir7_match/'
new_line = xt.line.Line.from_json(path_1+"b4_sequence.json")

Loading line from dict:   0%|          | 0/147676 [00:00<?, ?it/s]

Done loading line from dict.           


In [10]:
new_line.vars.get_table()

Table: 3125 rows, 2 cols
name                     value
t_turn_s                     0
version                  50801
pi                     3.14159
twopi                  6.28319
degrad                 57.2958
raddeg               0.0174533
e                      2.71828
amu0               1.25664e-06
emass              0.000510999
mumass                0.105658
nmass                 0.939565
umass                 0.931494
pmass                 0.938272
clight             2.99792e+08
qelect             1.60218e-19
hbar               6.58212e-25
erad               2.81794e-15
prad                1.5347e-18
none                         0
twiss_tol                1e-06
l.tan                      3.7
l.tanal                    3.7
l.tanar                    3.7
l.tanc                     3.7
l.tct                        1
l.tcth                       1
l.tctva                      1
l.mbas2                      0
l.mbaw                       0
l.mbcs2                      0
l.mbls2       

In [13]:
new_line.vars['kqtf.b2']._find_dependant_targets()

[vars['kqtf.b2'],
 vars['kqtf.a67b2'],
 element_refs['mqt.15r6.b2..2'].knl,
 element_refs['mqt.15r6.b2..2'].knl[1],
 element_refs['mqt.15r6.b2..2'],
 element_refs['mqt.15r6.b2..1'],
 element_refs['mqt.15r6.b2..1'].knl,
 element_refs['mqt.15r6.b2..1'].knl[1],
 element_refs['mqt.17r6.b2..2'].knl[1],
 element_refs['mqt.17r6.b2..2'].knl,
 element_refs['mqt.17r6.b2..2'],
 element_refs['mqt.17r6.b2..1'],
 element_refs['mqt.17r6.b2..1'].knl,
 element_refs['mqt.17r6.b2..1'].knl[1],
 element_refs['mqt.19r6.b2..2'].knl[1],
 element_refs['mqt.19r6.b2..2'],
 element_refs['mqt.19r6.b2..2'].knl,
 element_refs['mqt.19r6.b2..1'].knl[1],
 element_refs['mqt.19r6.b2..1'],
 element_refs['mqt.19r6.b2..1'].knl,
 element_refs['mqt.21r6.b2..2'],
 element_refs['mqt.21r6.b2..2'].knl[1],
 element_refs['mqt.21r6.b2..2'].knl,
 element_refs['mqt.21r6.b2..1'].knl[1],
 element_refs['mqt.21r6.b2..1'],
 element_refs['mqt.21r6.b2..1'].knl,
 element_refs['mqt.21l7.b2..2'],
 element_refs['mqt.21l7.b2..2'].knl,
 element_re

In [14]:
new_line['mqxfa.a1l1..1']

Multipole(_order=1, inv_factorial_order=1.0, length=0.2625, hxl=0.0, hyl=0.0, radiation_flag=0, delta_taper=0.0, knl=[0.         0.00149076], ksl=[0. 0.], _internal_record_id=RecordIdentifier(buffer_id=0, offset=0))

In [1]:
import numpy as np
import xtrack as xt

# We build a simple ring
pi = np.pi
lbend = 3
lquad = 0.3
elements = {
    'mqf.1': xt.Quadrupole(length=lquad, k1=0.1),
    'd1.1':  xt.Drift(length=1),
    'mb1.1': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd2.1':  xt.Drift(length=1),

    'mqd.1': xt.Quadrupole(length=lquad, k1=-0.7),
    'd3.1':  xt.Drift(length=1),
    'mb2.1': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd4.1':  xt.Drift(length=1),

    'mqf.2': xt.Quadrupole(length=lquad, k1=0.1),
    'd1.2':  xt.Drift(length=1),
    'mb1.2': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd2.2':  xt.Drift(length=1),

    'mqd.2': xt.Quadrupole(length=lquad, k1=-0.7),
    'd3.2':  xt.Drift(length=1),
    'mb2.2': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd4.2':  xt.Drift(length=1),
}
line = xt.Line(elements=elements, element_names=list(elements.keys()))
line.particle_ref = xt.Particles(p0c=1.2e9, mass0=xt.PROTON_MASS_EV)

In [6]:
line.vars['ciao!'] = 0
line.element_refs['mqf.1'].k1 = line.vars['ciao!'] / lquad

In [14]:
line.vars['ciao!']._get_value()

0

In [12]:
line['mqf.1'].k1 = 3

In [16]:
line['mqf.1'].k1

3.3333333333333335

In [15]:
line.vars['ciao!'] = 1

In [7]:
line.vars['ciao!']._find_dependant_targets()

[vars['ciao!'], element_refs['mqf.1'].k1, element_refs['mqf.1']]

In [17]:
line.vars.get_table()

Table: 2 rows, 2 cols
name     value
t_turn_s     0
ciao!        1

In [20]:
#For each quadrupole we create a variable controlling its integrated strength.
# Expressions can be associated to any beam element property, using the `element_refs`
# attribute of the line. For example:
line.vars['k1l.qf.1'] = 0
line.element_refs['mqf.1'].k1 = line.vars['k1l.qf.1'] / lquad
line.vars['k1l.qd.1'] = 0
line.element_refs['mqd.1'].k1 = line.vars['k1l.qd.1'] / lquad
line.vars['k1l.qf.2'] = 0
line.element_refs['mqf.2'].k1 = line.vars['k1l.qf.2'] / lquad
line.vars['k1l.qd.2'] = 0
line.element_refs['mqd.2'].k1 = line.vars['k1l.qd.2'] / lquad

In [21]:
# When a variable is changed, the corresponding element property is automatically
# updated:
line.vars['k1l.qf.1'] = 0.1
line['mqf.1'].k1 # is 0.333, i.e. 0.1 / lquad


0.33333333333333337

In [22]:
# We can create a variable controlling the integrated strength of the two
# focusing quadrupoles
line.vars['k1lf'] = 0.1
line.vars['k1l.qf.1'] = line.vars['k1lf']
line.vars['k1l.qf.2'] = line.vars['k1lf']
# and a variable controlling the integrated strength of the two defocusing quadrupoles
line.vars['k1ld'] = -0.7
line.vars['k1l.qd.1'] = line.vars['k1ld']
line.vars['k1l.qd.2'] = line.vars['k1ld']


In [23]:
# Changes on the controlling variable are propagated to the two controlled ones and
# to the corresponding element properties:
line.vars['k1lf'] = 0.2
line.vars['k1l.qf.1']._get_value() # is 0.2
line.vars['k1l.qf.2']._get_value() # is 0.2
line['mqf.1'].k1 # is 0.666, i.e. 0.2 / lquad
line['mqf.2'].k1 # is 0.666, i.e. 0.2 / lquad



0.6666666666666667

In [24]:
# The `_info()` method of a variable provides information on the existing relations
# between the variables. For example:
line.vars['k1l.qf.1']._info()

#  vars['k1l.qf.1']._get_value()
   vars['k1l.qf.1'] = 0.2

#  vars['k1l.qf.1']._expr
   vars['k1l.qf.1'] = vars['k1lf']

#  vars['k1l.qf.1']._expr._get_dependencies()
   vars['k1lf'] = 0.2

#  vars['k1l.qf.1']._find_dependant_targets()
   element_refs['mqf.1'].k1



In [25]:
line.vars['k1lf']._info()

#  vars['k1lf']._get_value()
   vars['k1lf'] = 0.2

#  vars['k1lf']._expr is None

#  vars['k1lf']._find_dependant_targets()
   vars['k1l.qf.2']
   element_refs['mqf.2'].k1
   vars['k1l.qf.1']
   element_refs['mqf.1'].k1



In [26]:
line.element_refs['mqf.1'].k1._info()


#  element_refs['mqf.1'].k1._get_value()
   element_refs['mqf.1'].k1 = 0.6666666666666667

#  element_refs['mqf.1'].k1._expr
   element_refs['mqf.1'].k1 = (vars['k1l.qf.1'] / 0.3)

#  element_refs['mqf.1'].k1._expr._get_dependencies()
   vars['k1l.qf.1'] = 0.2

#  element_refs['mqf.1'].k1 does not influence any target



In [27]:
# Expressions can include multiple variables and mathematical operations. For example
line.vars['a'] = 3 * line.functions.sqrt(line.vars['k1lf']) + 2 * line.vars['k1ld']

In [28]:
# As seen above, line.vars['varname'] returns a reference object that
# can be used to build further references, or to inspect its properties.
# To get the current value of the variable, one needs to use `._get_value()`
# For quick access to the current value of a variable, one can use the `line.varval`
# attribute or its shortcut `line.vv`:
line.varval['k1lf'] # is 0.2

0.2

In [29]:
line.vv['k1lf']     # is 0.2

0.2

In [30]:
# Note an important difference when using `line.vars` or `line.varval` in building
# expressions. For example:
line.vars['a'] = 3.
line.vars['b'] = 2 * line.vars['a']


In [31]:
# In this case the reference to the quantity `line.vars['a']` is stored in the
# expression, and the value of `line.vars['b']` is updated when `line.vars['a']`
# changes:
line.vars['a'] = 4.
line.vv['b'] # is 8.


8.0

In [32]:
# On the contrary, when using `line.varval` or `line.vv` in building expressions,
# the current value of the variable is stored in the expression:
line.vv['a'] = 3.
line.vv['b'] = 2 * line.vv['a']
line.vv['b'] # is 6.


6.0

In [33]:
line.vv['a'] = 4.
line.vv['b'] # is still 6.

6.0

In [34]:
# The `line.vars.get_table()` method returns a table with the value of all the
# existing variables:
line.vars.get_table()

Table: 10 rows, 2 cols
name     value
t_turn_s     0
ciao!        1
k1l.qf.1   0.2
k1l.qd.1  -0.7
k1l.qf.2   0.2
k1l.qd.2  -0.7
k1lf       0.2
k1ld      -0.7
a            4
b            6

In [35]:
# Regular expressions can be used to select variables. For example we can select all
# the variables containing `qf` using the following:
var_tab = line.vars.get_table()
var_tab.rows['.*qf.*']

Table: 2 rows, 2 cols
name     value
k1l.qf.1   0.2
k1l.qf.2   0.2